In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
# import seaborn as sns
import numpy as np
# import torch
# import torch.distributions.categorical
import scipy
import scipy.stats
import math
from scipy.interpolate import make_interp_spline, BSpline


In [2]:
def adjust_qr(rx, rp, rz):
    
    rz_new, rp_new = np.unique(rx, return_counts=True)
    rp_new = rp_new * rp[0]
    #rz_new= np.arange(0, len(rp_new), 1)
    
    return rx, rp_new, rz_new

In [3]:
def VaR(z, p, lim):
    var = 0
    cum_p = 0
    for j, atom in enumerate(p):
        cum_p += atom
        if cum_p>= lim:
            var = z[j]
            break
    return var

def CVaR (z, p, var, lim):
    cvar = var
    expected_c = 0 
    for i, j in enumerate(p):
        if p[i] > 0:
            expected_c += j * max(0, z[i]-var)
    cvar+= 1/(1- alpha) * expected_c
    return cvar

def variance(z, p):
    average = np.average(z, weights=p)
    variance = np.average((z-average)**2, weights=p)
    return variance

def weighted_mean(z,p):
    return np.average(z, weights=p)

def prob_threshold(z, p, thresh):
    cum_p = 0
    for j, atom in enumerate(p):        
        if z[j]>= thresh:
            cum_p += atom
            
    return cum_p

In [4]:
def l2 (u_values, v_values, u_weights=None, v_weights=None, debug=False):
    u_values = np.asarray(u_values, dtype=float)
    v_values = np.asarray(v_values, dtype=float)
    u_weights = np.asarray(u_weights, dtype=float)
    v_weights = np.asarray(v_weights, dtype=float)
    
    u_sorter = np.argsort(u_values)
    v_sorter = np.argsort(v_values)

    all_values = np.concatenate((u_values, v_values))
    all_values.sort(kind='mergesort')

    # Compute the differences between pairs of successive values of u and v.
    deltas = np.diff(all_values)

    # Get the respective positions of the values of u and v among the values of
    # both distributions.
    u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right')
    v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')

    # Calculate the CDFs of u and v using their weights
    u_sorted_cumweights = np.concatenate(([0],np.cumsum(u_weights[u_sorter])))
    u_cdf = u_sorted_cumweights[u_cdf_indices] / u_sorted_cumweights[-1]

    v_sorted_cumweights = np.concatenate(([0],np.cumsum(v_weights[v_sorter])))
    v_cdf = v_sorted_cumweights[v_cdf_indices] / v_sorted_cumweights[-1]
    
    if debug:
        print("delta", deltas)
        print("cdf u:", u_cdf)
        print("cdf v:", v_cdf)
        print("square cdf u - cdf v", np.square(u_cdf - v_cdf))
        print("square diff * delta", np.multiply(np.square(u_cdf - v_cdf), deltas))
        print("sum( (square diff) *delta))", np.sum(np.multiply(np.square(u_cdf - v_cdf), deltas)))
    return np.sqrt(np.sum(np.multiply(np.square(u_cdf - v_cdf), deltas)))


## Graphs

##### DTMC

In [5]:
def dtmc_compute(idx, apdx, experiment_name, rep_type, show_graph=True, debug=True, print_metrics=False, shaded=False, y_level=0.3, y_padding=0.3, alpha=0.7):
    if debug:
        print('Reading file: '+file_path+experiment_name+'/'+experiment_name+'_distr_dtmc_'+alg_type+'_'+rep_type+apdx+'.csv')
    dist = pd.read_csv(file_path+experiment_name+'/'+experiment_name+'_distr_dtmc_'+alg_type+'_'+rep_type+apdx+'.csv')
    #dist.head(10)
    rx = dist['r'].to_numpy()
    rp = dist['p'].to_numpy()
    n = np.max(np.where(rp > 1e-20))
    rx = rx[0:n+1]
    rp = rp[0:n+1]

    var_dtmc = VaR(rx, rp, alpha)
    metrics = np.array([weighted_mean(rx, rp), variance(rx, rp), var_dtmc, CVaR(rx, rp, var_dtmc, alpha), prob_threshold(rx, rp, thresh)])
    # z = z[0:n+1]
    if debug:
        print(n)
        print(np.sum(rp), np.allclose(np.sum(rp), 1))
        print("max p:", np.max(rp))
    
    distr = scipy.stats.rv_discrete(values=(rx, rp))
    if show_graph:
        # h = plt.bar(rx, rp, alpha=0.5)
        fig = plt.figure(figsize=fig_size)

        if shaded:
            # plt.bar(rx, distr.pmf(rx), width=0.98, color='gray', alpha=0.45, linewidth=0,label='PDF')
            kernel = scipy.stats.gaussian_kde(rx, weights=rp)
            range = np.arange(0, n, 1)
            # plt.plot(range, kernel(range), label='gaussian PDF')
            # 300 represents number of points to make between T.min and T.max
            xnew = np.linspace(6, rx.max(), 300) 

            spl = make_interp_spline(rx, distr.pmf(rx), k=3)  # type: BSpline
            power_smooth = spl(xnew)
            idx_start = (np.abs(xnew - metrics[2])).argmin()
            plt.plot(xnew, power_smooth, color='k', alpha=0.8, label='Smoothed PDF')
            
            plt.fill_between(xnew[idx_start:], power_smooth[idx_start:], 0, color='gray', alpha=.6)
            
            plt.vlines(x=[metrics[0], metrics[2], metrics[3]], linestyles='dashed', ymin=0, ymax=1, color = 'k')
            
            plt.text(metrics[0]+y_padding, y_level+0.15, r'$\mathbb{E}$', fontsize=graphsize-10,  rotation=90, verticalalignment='center')
            plt.text(metrics[2]-y_padding-1.5, y_level, r'$\mathsf{VaR}_{'+str(alpha)+r'}$', fontsize=graphsize-10, rotation=90, verticalalignment='center')
            plt.text(metrics[3]+y_padding, y_level, r'$\mathsf{CVaR}_{'+str(alpha)+r'}$', fontsize=graphsize-10, rotation=90, verticalalignment='center')

        else:
            if (xmax-xmin)>400:
                h = plt.bar(rx, distr.pmf(rx), width=1, alpha=0.5, label='PDF')
            else:
                h = plt.bar(rx, distr.pmf(rx), alpha=0.5, label='PDF')
            kernel = scipy.stats.gaussian_kde(rx, weights=rp)
            range = np.arange(0, n, 0.1)
            plt.plot(range, kernel(range), label='Smoothed PDF')
        plt.xlim([xmin,xmax])
        plt.ylim([0,ymax])

        if not shaded:
            plt.legend(loc='upper right', fontsize=graphsize-10)
        plt.tick_params(axis='both', which='major', labelsize=graphsize-15)
        plt.ticklabel_format(axis='both')
        plt.xlabel('Value', fontsize=graphsize-10)
        plt.ylabel('Probability', fontsize=graphsize-10)
        # plt.grid(b=True, which='major', color='#666666', linestyle='-', linewidth=0.25)
        plt.tight_layout()
        plt.show()
    # plt.fill_between(range, kernel(range))

    if save_graphs:
        print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_dtmc_'+alg_type+'_'+rep_type+apdx+".png")
        fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_dtmc_'+alg_type+'_'+rep_type+apdx+".png", transparent=True, dpi=400)


    if debug or print_metrics:
        print(weighted_mean(rx, rp),",",variance(rx, rp),",",var_dtmc,",", CVaR(rx, rp, var_dtmc, alpha),",",prob_threshold(rx, rp, thresh))
    return rx,rp, metrics, distr

##### Expected Value Iteration

In [6]:
def exp_compute(idx, apdx, experiment_name, rep_type, show_graph=True, debug=True, print_metrics=False, elements=False, figure = None, smooth=False, alpha=0.7):
    if debug:
        print('Reading file: '+file_path+experiment_name+'/'+experiment_name+'_distr_exp_'+rep_type+apdx+'.csv')
    dist = pd.read_csv(file_path+experiment_name+'/'+experiment_name+'_distr_exp_'+rep_type+apdx+'.csv')
    #dist.head(10)
    rx_e = dist['r'].to_numpy()
    rp_e = dist['p'].to_numpy()
    z_e = dist['z'].to_numpy()
    if 'qr' in rep_type:
        if debug:
            print('converting distribution from qr')
        rx_e, rp_e, z_e = adjust_qr(rx_e, rp_e, z_e)
        # print(len(rx_e), rx_e)
        # print(len(rp_e), rp_e)
        # print(len(z_e), z_e)

    n = np.max(np.where(rp_e > 1e-8))
    rx_e = rx_e[0:n+1]
    rp_e = rp_e[0:n+1]
    z_e = z_e[0:n+1]
    
    if debug:
        print(len(dist.index))
        print(np.sum(rp_e), np.allclose(np.sum(rp_e), 1))
        print("max p:", np.max(rp_e))
    
    if show_graph:
        distr_e = scipy.stats.rv_discrete(values=(z_e, rp_e))
        # h = plt.bar(rx_e, rp_e, alpha=0.5)
        fig=plt.figure(figsize=fig_size)
        
        if not elements:
            h = plt.bar(z_e, rp_e, alpha=0.5, color='g', label='PDF')
        else:
            h = plt.bar(z_e, rp_e, alpha=0.8, width=0.3, color='g')
            plt.plot(z_e, rp_e, 'go')
            if not smooth:
                plt.bar(figure[0], figure[1].pmf(figure[0]), width=0.98, color='k', alpha=0.3, linewidth=0,label='PDF')
            else:
                # 30`0 represents number of points to make between T.min and T.max
                xnew = np.linspace(6, figure[0].max(), 300) 

                spl = make_interp_spline(figure[0], figure[1].pmf(figure[0]), k=3)  # type: BSpline
                power_smooth = spl(xnew)

                plt.plot(xnew, power_smooth, color='k', alpha=0.5, label='Smoothed PDF')
                plt.fill_between(xnew, power_smooth, 0, color='gray', alpha=.3)
        kernel = scipy.stats.gaussian_kde(z_e, weights=rp_e)
        if 'qr' in rep_type:
            range = z_e
        else:
            range = np.arange(0, z_e[-1]+z_e[1], z_e[1])

        if not elements:
            plt.plot(range, kernel(z_e), color='g', label='Smoothed PDF')
        
        plt.xlim([xmin,xmax])
        plt.ylim([0,ymax])

        if not elements:
            plt.legend(loc='upper right', fontsize=graphsize-10)

        plt.tick_params(axis='both', which='major', labelsize=graphsize-15)
        plt.ticklabel_format(axis='both')
        plt.xlabel('Value', fontsize=graphsize-10)
        plt.ylabel('Probability', fontsize=graphsize-10)
        # plt.grid(b=True, which='major', color='#666666', linestyle='-', linewidth=0.25)
        plt.tight_layout()
        plt.show()
    # plt.fill_between(range, kernel(range))

        if save_graphs:
            if not smooth:
                print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_exp_'+rep_type+apdx+".png")
                fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_exp_'+rep_type+apdx+".png", transparent=True, dpi=400)
            else:
                print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_exp_'+rep_type+apdx+'_smooth'+".png")
                fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_exp_'+rep_type+apdx+'_smooth'+".png", transparent=True, dpi=400)
                
    
    var_exp = VaR(z_e, rp_e, alpha)
    if debug or print_metrics:
        print(f"{weighted_mean(z_e, rp_e)},{variance(z_e, rp_e)},{var_exp},{CVaR(z_e, rp_e, var_exp, alpha)},{prob_threshold(z_e, rp_e, thresh)}")
    return z_e, rp_e, np.array([weighted_mean(z_e, rp_e), variance(z_e, rp_e), var_exp, CVaR(z_e, rp_e, var_exp, alpha), prob_threshold(z_e, rp_e, thresh)])

##### CVaR

In [7]:
def cvar_compute(idx, apdx, experiment_name, rep_type, show_graph=True, debug=True, print_metrics=False, elements=False, figure=None, smooth =False, alpha=0.7):
    if debug:
        print('Reading file: '+file_path+experiment_name+'/'+experiment_name+'_distr_cvar_'+rep_type+apdx+'.csv')
    dist = pd.read_csv(file_path+experiment_name+'/'+experiment_name+'_distr_cvar_'+rep_type+apdx+'.csv')
    #dist.head(10)
    rx_c = dist['r'].to_numpy()
    rp_c = dist['p'].to_numpy()
    z_c = dist['z'].to_numpy()
    if 'qr' in rep_type:
        if debug:
            print('converting distribution from qr')
        rx_c, rp_c, z_c = adjust_qr(rx_c, rp_c, z_c)
    n = np.max(np.where(rp_c > 1e-6))
    rx_c = rx_c[0:n+1]
    rp_c = rp_c[0:n+1]
    z_c = z_c[0:n+1]
    if debug:
        print(n)
        print(np.sum(rp_c), np.allclose(np.sum(rp_c), 1))
        print("max p:", np.max(rp_c))
    
    if show_graph:
        distr_c = scipy.stats.rv_discrete(values=(z_c, rp_c))
        # h = plt.bar(rx_c, rp_c, alpha=0.5)
        fig=plt.figure(figsize=fig_size)
            
        if not elements:
            h = plt.bar(z_c, rp_c, alpha=0.5, color='g', label='PDF')
        else:
            h = plt.bar(z_c, rp_c, alpha=0.8, width=0.3, color='g')
            plt.plot(z_c, rp_c, 'go')
            if not smooth:
                plt.bar(figure[0], figure[1].pmf(figure[0]), width=0.98, color='k', alpha=0.3, linewidth=0,label='PDF')
            else:
                # 30`0 represents number of points to make between T.min and T.max
                xnew = np.linspace(6, figure[0].max(), 300) 

                spl = make_interp_spline(figure[0], figure[1].pmf(figure[0]), k=3)  # type: BSpline
                power_smooth = spl(xnew)

                plt.plot(xnew, power_smooth, color='k', alpha=0.5, label='Smoothed PDF')
                plt.fill_between(xnew, power_smooth, 0, color='gray', alpha=.3)
        kernel = scipy.stats.gaussian_kde(z_c, weights=rp_c)
        if 'qr' in rep_type:
            range = z_c
        else:
            range = np.arange(0, z_c[-1]+z_c[1], z_c[1]) 
        
        if not elements:
            plt.plot(range, kernel(range), color='g', label='Smoothed PDF')
        plt.xlim([xmin,xmax])
        plt.ylim([0,ymax])

        if not elements:
            plt.legend(loc='upper right', fontsize=graphsize-10)
        plt.tick_params(axis='both', which='major', labelsize=graphsize-15)
        plt.ticklabel_format(axis='both')
        plt.xlabel('Value', fontsize=graphsize-10)
        plt.ylabel('Probability', fontsize=graphsize-10)
        # plt.grid(b=True, which='major', color='#666666', linestyle='-', linewidth=0.25)
        plt.tight_layout()
        plt.show()
        # plt.fill_between(range, kernel(range))

        if save_graphs:
           
            if not smooth:
                print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_cvar_'+rep_type+apdx+".png")
                fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_cvar_'+rep_type+apdx+".png", transparent=True, dpi=400)
            else:
                print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_cvar_'+rep_type+apdx+'_smooth'+".png")
                fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_cvar_'+rep_type+apdx+'_smooth'+".png", transparent=True, dpi=400)

    var_cvar = VaR(z_c, rp_c, alpha)
    if debug or print_metrics:
        print(f"{weighted_mean(z_c, rp_c)},{variance(z_c, rp_c)},{var_cvar},{CVaR(z_c, rp_c, var_cvar, alpha)},{prob_threshold(z_c, rp_c, thresh)}")
    return z_c, rp_c, np.array([weighted_mean(z_c, rp_c), variance(z_c, rp_c), var_cvar, CVaR(z_c, rp_c, var_cvar, alpha), prob_threshold(z_c, rp_c, thresh)])

#### Setup

Set the metric parameters and graph parameters in next cell.

In [8]:
# Cvar alpha parameter
alpha=0.9 # 0.7 except for ds_treasure, betting_game, grid 150 -> 0.8
# threshold value when computing 
# the probability of a cost higher than the set threshold
thresh=30 
# graph figure parameter
xmax = 300
xmin=-1
ymax=0.1
fig_size= (10,5)

Set experiment name and representation type in the following cell.
Make sure to run the experiments using `run_experiments.py` before visualization.

In [9]:
# Graph Formating
matplotlib.rcParams.update({
    'font.family': 'serif',
    # 'text.usetex': False,
    # 'text.latex.preamble': r'\usepackage{amsfonts}',
    'axes.spines.right':False,
    'axes.spines.top': False
})
# font size in the graph
graphsize = 38
# experiments folder path
file_path = 'tests/experiments/'
experiment_name='gridworld_32'
apdx =  [''] 
alg_type = 'cvar' # Used only for the DTMC graph
rep_type = 'c51' # 'qr' or 'c51'
save_graphs = False # if set to true, save the graph to a png file
num_metrics = 5 # exp val, variance, var, cvar, prob>thresh

In [10]:
metrics_dtmc = np.zeros([len(apdx), num_metrics])
metrics_e = np.zeros([len(apdx), num_metrics])
metrics_c = np.zeros([len(apdx), num_metrics])
print_metrics=True

##### Base experiment Visualization
This cell shows the graph for the base experiment for the selected algorithms followed by a set of distributional metrics in the following order:
* Expected Value
* Variance
* Value at Risk(alpha)
* Conditional Value at Risk(alpha)
* Probability of the cost to be greater than a threshold.

['vi', 'exp', 'cvar'] are the possible options for the DTMC graph (in `algs`)

In [42]:
algs= ['exp','cvar']
show_graph = False
appendix  = '_less_atoms_2'
print('---DTMC---\n')
for idx, alg_type in enumerate(algs):
    _,_,_,_ = dtmc_compute(idx, appendix, experiment_name, rep_type, alpha=alpha, show_graph=show_graph, debug=False, print_metrics=True)
print('---DVI---')
_,_,_ = exp_compute(idx, appendix, experiment_name, rep_type, alpha=alpha, show_graph=show_graph, debug=False, print_metrics=True)
_,_,_ = cvar_compute(idx, appendix, experiment_name, rep_type, alpha=alpha, show_graph=show_graph, debug=False, print_metrics=True)

---DTMC---

35.90440147623608 , 5.411799120386391 , 39 , 41.1365098135993 , 1.0000000000000007
39.18472754506599 , 0.8336354992828318 , 39 , 40.847275450659936 , 1.0000000000000013
---DVI---
35.901064413773554,5.311689853231868,39.0,41.103403631105955,1.0000000000000004
39.1832629344655,0.7957549581344252,39.0,40.83262934465517,1.0000000000000007


##### Varying alpha cvar DVI

In [15]:
alpha_vals = [0.1, 0.5, 0.99]

algs = ['cvar']
rep_type = 'c51'
VaRs_approx = np.zeros([len(algs), len(alpha_vals)])
VaRs_dtmc = np.zeros([len(algs), len(alpha_vals)])
appendix = '_alpha_'

for idx, alg_type in enumerate(algs):

    print('\n--------------- '+ alg_type +'-------------\n')

    for i, a in enumerate(alpha_vals):

        rx_dtmc, rp_dtmc, _, _ = dtmc_compute(idx, appendix+str(a), experiment_name, rep_type, alpha=0, show_graph=False, debug=False, print_metrics=False)

        if 'exp' in alg_type:
            rx_approx, rp_approx, _ = exp_compute(idx, appendix+str(a), experiment_name, rep_type, alpha=0, show_graph=False, debug=False, print_metrics=False)
        elif 'cvar' in alg_type:
            rx_approx, rp_approx, _ = cvar_compute(idx, appendix+str(a), experiment_name, rep_type, alpha=0, show_graph=False, debug=False, print_metrics=False)

        
        VaRs_dtmc[idx][i] = VaR(rx_dtmc, rp_dtmc, a)
        VaRs_approx [idx][i] = VaR(rx_approx, rp_approx, a)
        print(f'alpha:{a:0.2f}, \t{VaRs_approx[idx][i]:0.2f}, \t dtmc:{VaRs_dtmc[idx][i]:0.2f}')


--------------- cvar-------------

alpha:0.10, 	42.00, 	 dtmc:44.00


FileNotFoundError: [Errno 2] No such file or directory: 'tests/experiments/selfStabilising_15/selfStabilising_15_distr_dtmc_cvar_c51_alpha_0.5.csv'

##### Varying alpha for one exp and cvar DVI

In [80]:
alphas = np.append(np.arange(0.1,1,0.1), 0.99)

algs = ['exp', 'cvar']
rep_type = 'c51'
VaRs_approx = np.zeros([len(algs), len(alphas)])
VaRs_dtmc = np.zeros([len(algs), len(alphas)])
appendix  = ''


for idx, alg_type in enumerate(algs):

    print('\n--------------- '+ alg_type +'-------------\n')

    rx_dtmc, rp_dtmc, _, _ = dtmc_compute(idx, appendix, experiment_name, rep_type, alpha=0, show_graph=False, debug=False, print_metrics=False)

    if 'exp' in alg_type:
        rx_approx, rp_approx, _ = exp_compute(idx, appendix, experiment_name, rep_type, alpha=0, show_graph=False, debug=False, print_metrics=False)
    elif 'cvar' in alg_type:
        rx_approx, rp_approx, _ = cvar_compute(idx, appendix, experiment_name, rep_type, alpha=0, show_graph=False, debug=False, print_metrics=False)

    for i, a in enumerate(alphas):
        VaRs_dtmc[idx][i] = VaR(rx_dtmc, rp_dtmc, a)
        VaRs_approx [idx][i] = VaR(rx_approx, rp_approx, a)
        print(f'alpha:{a:0.2f}, \t{VaRs_approx[idx][i]:0.2f}, \t dtmc:{VaRs_dtmc[idx][i]:0.2f}')




--------------- exp-------------

alpha:0.10, 	39.00, 	 dtmc:42.00
alpha:0.20, 	54.00, 	 dtmc:55.00
alpha:0.30, 	66.00, 	 dtmc:67.00
alpha:0.40, 	78.00, 	 dtmc:79.00
alpha:0.50, 	90.00, 	 dtmc:92.00
alpha:0.60, 	105.00, 	 dtmc:106.00
alpha:0.70, 	126.00, 	 dtmc:124.00
alpha:0.80, 	150.00, 	 dtmc:147.00
alpha:0.90, 	180.00, 	 dtmc:185.00
alpha:0.99, 	222.00, 	 dtmc:303.00

--------------- cvar-------------

alpha:0.10, 	48.00, 	 dtmc:49.00
alpha:0.20, 	57.00, 	 dtmc:60.00
alpha:0.30, 	69.00, 	 dtmc:69.00
alpha:0.40, 	78.00, 	 dtmc:80.00
alpha:0.50, 	90.00, 	 dtmc:91.00
alpha:0.60, 	105.00, 	 dtmc:104.00
alpha:0.70, 	120.00, 	 dtmc:121.00
alpha:0.80, 	144.00, 	 dtmc:143.00
alpha:0.90, 	180.00, 	 dtmc:179.00
alpha:0.99, 	297.00, 	 dtmc:293.00


##### Running varying atoms
Print the distributional metrics for :

* metrics for risk-neutral DVI distributions 
* metrics for DTMC distributions induced from risk-neutral DVI policies

In [12]:
# apdx = ['_1','_10', '_25', '_50', '_75', '_100']
apdx = ['_1','_10', '_25', '_50', '_75', '_100','_500', '_1000', '_5000']
rep_type = 'c51' # c51 or qr
metrics_dtmc = np.zeros([len(apdx), num_metrics])
metrics_e = np.zeros([len(apdx), num_metrics])
metrics_c = np.zeros([len(apdx), num_metrics])
print_metrics=True

In [13]:
for idx, apd in enumerate(apdx):
    if 'c51' in rep_type and idx > 5:
        continue
    _,_, metrics_e[idx] = exp_compute(idx, apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False, print_metrics=print_metrics)

22.198584820082953,1727.0813139938775,0.0,73.99528273360986,0.22198584820082956
25.183306629639333,337.19490128877686,30.0,47.364294542913214,0.44154180891252176
25.20655577592385,237.1664364291676,32.0,44.27235091183038,0.31155448164018656
25.217091685282934,203.97988126033346,32.0,43.538645632546704,0.31877582133666044
25.217822134247456,192.6263956930828,32.0,43.3465599435923,0.31938448887841997
25.222438783925277,187.6615860610359,34.0,43.255809433956706,0.3221531929107059


In [14]:
alg_type = 'exp'
for idx, apd in enumerate(apdx):
    if 'c51' in rep_type and idx > 5:
        continue
    _,_, metrics_dtmc[idx],_ = dtmc_compute(idx, apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False, print_metrics=print_metrics)

25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217
25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217
25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217
25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217
25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217
25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217


##### Running varying slack variable atoms
Print the distributional metrics for :

* metrics for risk-sensitive DVI distributions 
* metrics for DTMC distributions induced from risk-sensitive DVI policies

In [15]:
if 'c51' in rep_type:
    apdx =['_0', '_10', '_25', '_50', '_75', '_100']
for idx, apd in enumerate(apdx):
    if idx > 5:
        continue
    _,_, metrics_c[idx] = cvar_compute(idx, apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False, print_metrics=print_metrics)

25.22715220687835,188.3107318723277,34.0,43.29079153403888,0.3213522374149526
25.256090875275884,185.3028618319587,34.0,43.218120777572054,0.31735140387614497
26.536658982523345,141.07424047715608,25.0,41.76424126829117,0.2600821561756963
27.12318252051541,91.46326063138912,25.0,37.8645843683783,0.17976812513208237
27.238398796080578,77.77733178037639,26.0,36.727134093652495,0.16334944527954892
27.32264038620908,73.86729769078296,27.0,36.5156398196263,0.1659164565936778


In [16]:
alg_type = 'cvar'
for idx, apd in enumerate(apdx):
    if 'c51' in rep_type and idx > 5:
        continue
    _,_, metrics_dtmc[idx],_ = dtmc_compute(idx, apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False, print_metrics=print_metrics)

25.240120673780105 , 190.6083703184246 , 34 , 43.33401975704474 , 0.32135223741495217
25.268647920821824 , 187.52609010863205 , 34 , 43.25997759605853 , 0.31735140387614513
26.54377081354217 , 142.3019641001608 , 25 , 41.78794737168728 , 0.26008215617569636
27.126541712767445 , 92.03302032468598 , 25 , 37.87578167588508 , 0.1797681251320825
27.240899948786005 , 78.20121787319654 , 26 , 36.73547126933729 , 0.16334944527954903
27.32486329474862 , 74.24472872117421 , 27 , 36.523049514758135 , 0.1659164565936776


##### Distributional distance between DTMC and DVI distributions

This goes through the experiment which varies the distributional approximation for either risk neutral ('exp') or risk-sensitive DVI ('cvar').

* `l2` : denotes the Cramer distributional distance
* `w1`: denotes 1-Wasserstein distributional distance


In [17]:

apdx = ['_1','_10', '_25', '_50', '_75', '_100','_500', '_1000', '_5000']

alg_type = 'exp' # Set to 'exp' or 'cvar'
rep_types = [ 'c51'] # c51 or qr
experiment_name='gridmap_10'

metrics_dtmc = np.zeros([len(apdx), num_metrics])
metrics_e = np.zeros([len(apdx), num_metrics])
metrics_c = np.zeros([len(apdx), num_metrics])
print_metrics=True

distance = np.zeros([len(apdx)])
distance2 =np.zeros([len(apdx)])

if 'cvar' in alg_type:
    apdx = [el+'_va' for el in apdx]

for rep_type in rep_types:
    print(rep_type)
    if 'c51' in rep_type:
        print('----- l2-----')
    elif 'qr' in rep_type:
        print('----- w1-----')

    for idx, apd in enumerate(apdx):
        
        if 'c51' in rep_type and idx > 5:
            continue
        rx, rp, metrics_dtmc[idx],_ = dtmc_compute(idx,  apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False)

        if 'exp' in alg_type:
            rx_v, rp_v, metrics_e[idx] = exp_compute(idx, apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False)
        else:
            rx_v, rp_v, metrics_c[idx] = cvar_compute(idx, apd, experiment_name, rep_type, alpha=alpha, show_graph=False, debug=False)

        if 'c51' in rep_type:
            distance[idx] = l2(rx, rx_v, rp, rp_v, debug=False)

        elif 'qr' in rep_type:
            distance[idx] = scipy.stats.wasserstein_distance(rx, rx_v, rp, rp_v)
            distance2[idx] = l2(rx, rx_v, rp, rp_v, debug=False)
            
        print(distance[idx])
    
    if 'qr' in rep_type:
        print('----- l2-----')
        for idx in range(len(apdx)):
            print(distance2[idx])

c51
----- l2-----
3.5091899375252718
1.0212876684802066
0.6678529169909043
0.4219101200524545
0.2510191310106609
0.003986024765100188


### Debugging

#### Example figure

 public double [] update_support(double gamma, double state_reward, double []sum_p){

        double [] m = new double [atoms];
        double b = 0;
        for (int j =0; j<atoms; j++){
            double temp = max(v_min, min(v_max, state_reward+gamma*z[j]));
            if (delta_z > 0.0) {
                 b = (temp - v_min)/delta_z;
            }
            int l= (int) floor(b); int u= (int) ceil(b);

            if ( l- u != 0){
                m[l] += sum_p[j] * (u -b);
                m[u] += sum_p[j] * (b-l);
            } else{
                m[l] += sum_p[j];
            }
        }

        return m;
    }

In [11]:
def get_support(arr_p, rz):

    m = np.zeros([atoms])
    b = 0
    vmin = rz[0]
    delta_z = abs(rz[1] - rz[0])
    atoms = len(rz)
    assert delta_z >0 or atoms ==0

    for j in range(atoms):
        b = (rz[j] - vmin) / delta_z
        l= int(math.floor(b)); u= int(math.ceil(b));
    
        if ( l- u != 0):
                m[l] += arr_p[j] * (u -b);
                m[u] += arr_p[j] * (b-l);
        else:
                m[l] += arr_p[j];
    
    return m

def plot_distr(rx_c, rp_c, z_c, apdx, experiment_name, rep_type, show_graph=True, debug=True, print_metrics=False, elements=False, figure=None, smooth =False, alpha=0.7):
    n = np.max(np.where(rp_c > 1e-6))
    rx_c = rx_c[0:n+1]
    rp_c = rp_c[0:n+1]
    z_c = z_c[0:n+1]
    if debug:
        print(n)
        print(np.sum(rp_c), np.allclose(np.sum(rp_c), 1))
        print("max p:", np.max(rp_c))
    
    if show_graph:
        distr_c = scipy.stats.rv_discrete(values=(z_c, rp_c))
        # h = plt.bar(rx_c, rp_c, alpha=0.5)
        fig=plt.figure(figsize=fig_size)
            
        if not elements:
            h = plt.bar(z_c, rp_c, alpha=0.5, color='g', label='PDF')
        else:
            h = plt.bar(z_c, rp_c, alpha=0.8, width=0.3, color='g')
            plt.plot(z_c, rp_c, 'go')
            if not smooth:
                plt.bar(figure[0], figure[1].pmf(figure[0]), width=0.98, color='k', alpha=0.3, linewidth=0,label='PDF')
            else:
                # 30`0 represents number of points to make between T.min and T.max
                xnew = np.linspace(6, figure[0].max(), 300) 

                spl = make_interp_spline(figure[0], figure[1].pmf(figure[0]), k=3)  # type: BSpline
                power_smooth = spl(xnew)

                plt.plot(xnew, power_smooth, color='k', alpha=0.5, label='Smoothed PDF')
                plt.fill_between(xnew, power_smooth, 0, color='gray', alpha=.3)
        kernel = scipy.stats.gaussian_kde(z_c, weights=rp_c)
        if 'qr' in rep_type:
            range = z_c
        else:
            range = np.arange(0, z_c[-1]+z_c[1], z_c[1]) 
        
        if not elements:
            plt.plot(range, kernel(range), color='g', label='Smoothed PDF')
        plt.xlim([xmin,xmax])
        plt.ylim([0,ymax])

        if not elements:
            plt.legend(loc='upper right', fontsize=graphsize-10)
        plt.tick_params(axis='both', which='major', labelsize=graphsize-15)
        plt.ticklabel_format(axis='both')
        plt.xlabel('Value', fontsize=graphsize-10)
        plt.ylabel('Probability', fontsize=graphsize-10)
        # plt.grid(b=True, which='major', color='#666666', linestyle='-', linewidth=0.25)
        plt.tight_layout()
        plt.show()
        # plt.fill_between(range, kernel(range))

        if save_graphs:
            if not smooth:
                print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_cvar_fake_'+rep_type+apdx+".png")
                fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_cvar_fake_'+rep_type+apdx+".png", transparent=True, dpi=400)
            else:
                print('Saving to:'+file_path+experiment_name+'/'+experiment_name+'_distr_cvar_fake_'+rep_type+apdx+'_smooth'+".png")
                fig.savefig(file_path+experiment_name+'/'+experiment_name+'_distr_cvar_fake_'+rep_type+apdx+'_smooth'+".png", transparent=True, dpi=400)

    var_cvar = VaR(z_c, rp_c, alpha)
    if debug or print_metrics:
        print(f"{weighted_mean(z_c, rp_c)},{variance(z_c, rp_c)},{var_cvar},{CVaR(z_c, rp_c, var_cvar, alpha)},{prob_threshold(z_c, rp_c, thresh)}")
    return z_c, rp_c, np.array([weighted_mean(z_c, rp_c), variance(z_c, rp_c), var_cvar, CVaR(z_c, rp_c, var_cvar, alpha), prob_threshold(z_c, rp_c, thresh)])

In [12]:

xmax = 25
algs=['cvar'] #['vi', 'exp', 'cvar']
smoothed= True

print('---DTMC---')
for idx, alg_type in enumerate(algs):
    rx,_,_,dist = dtmc_compute(idx, '_ex_fig_2', 'test', 'c51', alpha=alpha, show_graph=True, debug=False, print_metrics=True, shaded=True, y_padding=0.28)

print('---DVI---')
if 'exp' in algs:
    _,_,_ = exp_compute(idx, '_ex_fig_2', 'test', 'c51', alpha=alpha, show_graph=True, debug=False, print_metrics=True, elements=True, figure=(rx, dist), smooth=smoothed)
    _,_,_ = exp_compute(idx, '_ex_fig_2', 'test', 'qr', alpha=alpha, show_graph=True, debug=False, print_metrics=True, elements=True, figure=(rx, dist), smooth=smoothed)
else:
    rz,_,_ = cvar_compute(idx, '_ex_fig_2', 'test', 'c51', alpha=alpha, show_graph=True, debug=False, print_metrics=True, elements=True, figure=(rx, dist), smooth=smoothed)
    _,_,_ = cvar_compute(idx, '_ex_fig_2', 'test', 'qr', alpha=alpha, show_graph=True, debug=False, print_metrics=True, elements=True, figure=(rx, dist), smooth=smoothed)

m = get_support(dist, rz)

_,_,_ = plot_distr(rx, m, rz, '_ex_fig_2', 'test', 'c51', alpha=alpha, show_graph=True, debug=False, print_metrics=True, elements=True, figure=(rx, dist), smooth=smoothed)



---DTMC---


FileNotFoundError: [Errno 2] No such file or directory: 'tests/experiments/test/test_distr_dtmc_cvar_c51_ex_fig_2.csv'